# Atomic Agents × Opik – Quick-Start

A minimal, self-contained example demonstrating how to trace Atomic-Agents workflows

with Opik.

## What is Atomic Agents?

**Atomic Agents** is a lightweight Python framework for building *agentic* applications – think of

it as a minimal alternative to LangChain or LangGraph that focuses on composability and clear

APIs.  An `Agent` encapsulates your business logic, while `Tool` objects expose

re-usable functions your agents (or humans) can call.

## Installation

Install the required libraries (choose a virtual-env of your preference):

```bash

pip install "opik[atomic_agents]" atomic-agents instructor

```



* `opik[atomic_agents]` – Opik Python SDK with the Atomic-Agents extra.

* `atomic-agents` – the framework itself.

* `instructor` – micro-library we use to stub a non-networking LLM client (feel free to replace).

In [1]:
from pydantic import Field
from instructor.client import Instructor

# Atomic-Agents runtime
from atomic_agents.agents.base_agent import (
    BaseAgent as BaseChatAgent,
    BaseAgentConfig as BaseChatAgentConfig,
)

# Opik integration helpers
from opik.integrations.atomic_agents import track_atomic_agents, OpikContextProvider
from opik import context_storage


And now, with just a simple one-liner, all the `BaseAgent` interactions will be logged to Opik...

In [2]:
track_atomic_agents(project_name="atomic-quickstart")


In [3]:
from atomic_agents.lib.base.base_io_schema import BaseIOSchema

class ChatMessage(BaseIOSchema):
    """A simple chat message for input and output"""
    chat_message: str = Field(..., description="User message")


class EchoAgent(BaseChatAgent):
    """Agent that echoes back the input message."""

    input_schema = ChatMessage
    output_schema = ChatMessage



In [4]:
# Create dummy client that returns an echo response
dummy_instructor_client = Instructor(
    client=None, 
    create=lambda **kwargs: ChatMessage(chat_message="Echo: Hello Atomic Agents!")
)

agent = EchoAgent(
    config=BaseChatAgentConfig(model="gpt-3.5-turbo", client=dummy_instructor_client)
)

result = agent.run(ChatMessage(chat_message="Hello Atomic Agents!"))
print("Agent result:", result)


OPIK: ========================
The API key must be specified to log data to https://www.comet.com/opik.
You can use `opik configure` CLI command to configure your environment for logging.
See the configuration details in the docs: https://www.comet.com/docs/opik/tracing/sdk_configuration.


OPIK: Started logging traces to the "atomic-quickstart" project at https://www.comet.com/opik/api/v1/session/redirect/projects/?trace_id=019828c1-77a8-7437-9508-7ac93f61c504&path=aHR0cHM6Ly93d3cuY29tZXQuY29tL29waWsvYXBpLw==.



Agent result: {"chat_message":"Echo: Hello Atomic Agents!"}


In [ ]:
ctx = context_storage.get_current_context_instance()
trace_data = ctx.get_trace_data()
if trace_data and hasattr(trace_data, 'id'):
    trace_id = trace_data.id
    print("Opik trace_id:", trace_id)
    provider = OpikContextProvider(project_name="atomic-quickstart", trace_id=trace_id)
    print("\nProvider renders:\n", provider.get_info())
else:
    print("No active trace found - this may mean tracing is not properly configured")
    provider = OpikContextProvider(project_name="atomic-quickstart", trace_id="<no-active-trace>")
    print("\nProvider renders:\n", provider.get_info())


No active trace found - this may mean tracing is not properly configured

Provider renders:
 [Opik Trace Info]
trace_id: <no-active-trace>
project:  atomic-quickstart


OPIK: Failed to process CreateSpansBatchMessage. Error: headers: {'date': 'Sun, 20 Jul 2025 16:53:46 GMT', 'content-type': 'application/json', 'content-length': '51', 'connection': 'keep-alive', 'set-cookie': 'AWSALB=wFoVwB2GntoErCHBfPJnzpOrf1F8E1zmz5oGcRrFs4Sj/g2sP1I8le+qYlI5z+M7QUhEaVT+nDrmO0cj9hPygB7VNVB2E4uV0QASqSWJACiI/p/fvIXV+u8ZSRII; Expires=Sun, 27 Jul 2025 16:53:46 GMT; Path=/, AWSALBCORS=wFoVwB2GntoErCHBfPJnzpOrf1F8E1zmz5oGcRrFs4Sj/g2sP1I8le+qYlI5z+M7QUhEaVT+nDrmO0cj9hPygB7VNVB2E4uV0QASqSWJACiI/p/fvIXV+u8ZSRII; Expires=Sun, 27 Jul 2025 16:53:46 GMT; Path=/; SameSite=None; Secure', 'server': 'nginx'}, status_code: 401, body: {'code': 401, 'message': 'API key should be provided'}
OPIK: Failed to process CreateTraceBatchMessage. Error: headers: {'date': 'Sun, 20 Jul 2025 16:53:46 GMT', 'content-type': 'application/json', 'content-length': '51', 'connection': 'keep-alive', 'set-cookie': 'AWSALB=tVFtAKolwZSSWsiw+U4ChGA+z1lIAtXECvbiv96tdbX1kXN+8xj+hQRoc6Zd0O0GALgB7tVa6SV9n913Lqx3Z+

## Visualising your trace

1. Export your Opik API key & workspace URL, e.g.

   ```bash

   export OPIK_API_KEY="..."

   export OPIK_BASE_URL="https://app.opik.ai"

   ```

2. Run this script.  Head to *Traces → atomic-quickstart* and you’ll see the full hierarchy:

   * root **Agent** span

   * nested **LLM** span (BaseChatAgent `get_response`)

   * any **Tool** spans you might call



### Next steps

* Add real `BaseTool` subclasses and watch them appear in the timeline.

* Pass `metadata={...}` or `tags=[...]` into `track_atomic_agents` for richer filtering.

* Use `OpikContextProvider` inside your prompts so downstream agents can reference the

  current `trace_id` (see the other tutorial).